In [24]:
import random
from collections import Counter
import itertools

In [26]:
def hammingDistance(p, q):
    count = 0
    for i in range(len(p)):
        if p[i] != q[i]:
            count += 1
    return count

In [20]:
def profile_probability(consensus, profile):
    prob = 1
    for i in range(len(consensus)):
        prob *= profile[consensus[i]][i]
    return prob

In [22]:
def profile_to_consensus(profile):
    
    nu_list = ['A','C','G','T']
    nu_candidates = []
    consensus_list = []
    for i in range(len(profile['A'])):
        current_col = [profile[x][i] for x in nu_list]
        max_freq = max(enumerate(current_col),key=lambda x: x[1])[1]
        current_candidates = []
        for j in range(len(current_col)):
            if current_col[j] == max_freq:
                current_candidates.append(nu_list[j])
        nu_candidates.append(current_candidates)
    #print(nu_candidates)
    for x in list(itertools.product(*nu_candidates)):
        consensus_list.append(''.join(y for y in x))
    return consensus_list

In [3]:
def create_profile(motifs):
    col_len = len(motifs)
    A_list = []
    C_list = []
    G_list = []
    T_list = []
    for i in range(len(motifs[0])):
        col_nucleotides = [each_motif[i] for each_motif in motifs]
        c = Counter(col_nucleotides)
        A_list.append(c['A']/col_len)
        C_list.append(c['C']/col_len)
        G_list.append(c['G']/col_len)
        T_list.append(c['T']/col_len)
    profile = {'A':A_list, 'C':C_list, 'G':G_list, 'T':T_list}
    return profile

In [30]:
def create_profile_laplacian(motifs):
    col_len = len(motifs)
    A_list = []
    C_list = []
    G_list = []
    T_list = []
    for i in range(len(motifs[0])):
        col_nucleotides = [each_motif[i] for each_motif in motifs]
        c = Counter(col_nucleotides)
        A_list.append((c['A']+1)/(2*col_len))
        C_list.append((c['C']+1)/(2*col_len))
        G_list.append((c['G']+1)/(2*col_len))
        T_list.append((c['T']+1)/(2*col_len))
    profile = {'A':A_list, 'C':C_list, 'G':G_list, 'T':T_list}
    return profile      

In [6]:
def profile_most_probable_kmer(dna, k, profile):
    prob = 0
    most_probable_kmer = ''
    for i in range(len(dna)-k+1):
        current_kmer = dna[i:i+k]
        if profile_probability(current_kmer, profile) > prob:
            prob = profile_probability(current_kmer, profile)
            most_probable_kmer = current_kmer 
            
    if prob == 0:
        most_probable_kmer = dna[:k]
    return most_probable_kmer

In [52]:
def score(motifs):
    profile = create_profile_laplacian(motifs)
    consensus = profile_to_consensus(profile)
    total_score = 0
    for i in motifs:
        total_score += hammingDistance(consensus, i)
    return total_score
    

In [50]:
def randomizedMotifSearch(dna, k, t):
    random_indexes = [random.randint(0, len(dna[0])-k) for _ in range(t)]
    #print(random_indexes)
    motifs = [dna[i][random_indexes[i]:random_indexes[i]+k] for i in range(len(random_indexes))]
    bestMotifs = motifs
    while True:
        profile = create_profile_laplacian(motifs)
        motifs = [profile_most_probable_kmer(x, k, profile) for x in dna]
        if score(motifs) < score(bestMotifs):
            bestMotifs = motifs
        else:
            return bestMotifs

In [36]:
sample_dna = ['CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA','GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG','TAGTACCGAGACCGAAAGAAGTATACAGGCGT','TAGATCAAGTTTCAGGTGCACGTCGGTGAACC','AATCCACCAGCTCCACGTGCAATGTTGGCCTA']

In [51]:
randomizedMotifSearch(sample_dna, 8, 5)

IndexError: string index out of range

In [44]:
score_list = []
bestMotifs_list = []
result_list = []
for i in range(1000):
    bestMotifs, best_score = randomizedMotifSearch(sample_dna, 8, 5)
    score_list.append(best_score)
    bestMotifs_list.append(bestMotifs)

min_score = min(score_list)
for i in range(score_list):
    if score_list[i] == min_score:
        result_list.append(bestMotifs[i])

return set(result_list)

TypeError: 'int' object is not callable